In [1]:
from inputs import get_gamepad
from jetracer.nvidia_racecar import NvidiaRacecar

In [2]:
car = NvidiaRacecar()
car.steering = 0
car.throttle_gain = 0.25

In [3]:
while 1:
    events = get_gamepad()
    for event in events:
        if event.ev_type == 'Absolute':
            if event.code == 'ABS_Y':
                car.throttle = -(event.state - 127.5) / 127.5
                #print(f'Y={event.state}')
            if event.code == 'ABS_Z':
                car.steering = (event.state - 127.5) / 127.5
                #print(f'Z={event.state}')
        if event.ev_type == 'Key':
            if event.code == 'BTN_TR' and event.state == 1:
                car.throttle_gain = min(1.0, car.throttle_gain + 0.05)
            if event.code == 'BTN_TL' and event.state == 1:
                car.throttle_gain = max(0.0, car.throttle_gain - 0.05)
            print(f"throttle_gain={car.throttle_gain:.1f}")
        #print(event.ev_type, event.code, event.state)

throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.4
throttle_gain=0.4
throttle_gain=0.4
throttle_gain=0.4
throttle_gain=0.4
throttle_gain=0.4
throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.3
throttle_gain=0.2
throttle_gain=0.2
throttle_gain=0.2
throttle_gain=0.2
throttle_gain=0.2
throttle_gain=0.2


KeyboardInterrupt: 